In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

# 1. Basic autograd example 1

Autograd adalah salah satu fitur inti dari PyTorch yang membuatnya menjadi framework yang sangat populer untuk penelitian dan pengembangan dalam bidang deep learning. 

Autograd adalah mesin diferensiasi otomatis yang memungkinkan PyTorch untuk menghitung gradien secara efisien dari fungsi yang kompleks, seperti yang umum ditemukan dalam jaringan saraf tiruan.

Diferensiasi mengacu pada proses menghitung gradien dari suatu fungsi. Gradien ini kemudian digunakan dalam algoritma optimasi seperti gradient descent untuk memperbarui parameter model secara iteratif sehingga model dapat belajar dari data.

Cara Kerja Autograd?

* Tensor: Semua tensor dalam PyTorch memiliki atribut requires_grad yang secara default bernilai False. Jika diubah menjadi True, PyTorch akan mulai melacak semua operasi yang dilakukan pada tensor tersebut.
* Computational Graph: Ketika Anda melakukan operasi pada tensor dengan requires_grad=True, PyTorch secara otomatis membangun sebuah computational graph. Graph ini merepresentasikan hubungan antara operasi-operasi yang dilakukan pada tensor tersebut.
* Backward Pass: Setelah melakukan forward pass (menghitung output dari model), Anda dapat memanggil metode .backward() pada tensor output untuk menghitung gradien dari semua tensor yang terlibat dalam computational graph.

In [2]:
# buat tensor dengan requires_grad= True 

x = torch.randn(3, requires_grad=True)
print(x)

tensor([-0.7273,  0.5686,  0.7155], requires_grad=True)


In [3]:
y = x * 2
z = y.mean()

In [4]:
z.backward() # for  requires_grad=False RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [5]:
print(x.grad)

tensor([0.6667, 0.6667, 0.6667])


In [6]:
# create tensors

x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

In [7]:
# build a computational graph

y = w*x*b

# compute gradients
y.backward()

In [8]:
# print out the gradients

print(x.grad)
print(w.grad)
print(b.grad)

tensor(6.)
tensor(3.)
tensor(2.)


# Basic autograd example 2


In [9]:
# create tensors of shape (10, 3) and (10, 2)
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# build a fully connected layer 
linear = nn.Linear(3, 2)
print('w: ', linear.weight)
print('b:', linear.bias)

w:  Parameter containing:
tensor([[ 0.0950,  0.3328, -0.1448],
        [-0.4615,  0.2062, -0.3463]], requires_grad=True)
b: Parameter containing:
tensor([0.2422, 0.5299], requires_grad=True)


In [10]:
# build loss function and optimizer

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr= 0.01)

In [11]:
# Forward pass (menghitung output model)

pred = linear(x)

In [12]:
# Compute loss 
loss = criterion(pred, y)
print('loss:', loss.item())

loss: 1.2291069030761719


In [13]:
# Backward pass
loss.backward()

In [14]:
# Print out the gradients
print ('dL/dw: ', linear.weight.grad)
print('dL/db:', linear.bias.grad)

dL/dw:  tensor([[ 0.3929,  0.5996,  0.3243],
        [-0.1792,  0.1416,  0.0720]])
dL/db: tensor([0.5514, 0.2776])


In [15]:
# 1-step gradient descent
optimizer.step()

In [16]:
# you can also perform gradient descent at the low level 
# linear.weight.data.sub(0.01 * linear.weight.grad.data)
# linear.bias.data.sub(0.01 * linear.bias.grad.data)

# print out the loss after 1-step gradient descent
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization:', loss.item())

loss after 1 step optimization: 1.21860671043396


# 3. Loading data from numpy 

In [17]:
# create a numpy array
x = np.array([[1, 2], [3,4]])
print(x)

[[1 2]
 [3 4]]


In [18]:
# convert the numpy array to a torch tensor
y = torch.from_numpy(x)
print(y)

tensor([[1, 2],
        [3, 4]], dtype=torch.int32)


In [19]:
# convert the torch tensor to a numpy array
z = y.numpy()
print(z)

[[1 2]
 [3 4]]


# 4. Input Pipeline

In [20]:
# download and construct CIFAR-10 dataset

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

Files already downloaded and verified


In [21]:
# fetch one data pair (read data from disk)
image, label = train_dataset[0]
print(image.size())
print(label)

torch.Size([3, 32, 32])
6


In [22]:
# data loader (this provides queueu and threads in a very simple way)
train_loader = torch.utils.data.DataLoader(dataset= train_dataset, batch_size=64, shuffle= True)


In [23]:
# when iteration starts, queue and thread start to load data from files
data_iter = iter(train_loader)

In [24]:
from torch.utils.data import DataLoader

# mini batch images and labels
for images, labels in data_iter:
    pass

# 5. Input pipeline for custom dataset

In [25]:
import os
from PIL import Image  # Assuming PIL for image processing
from torch.utils.data import Dataset, DataLoader

In [26]:
import os
from PIL import Image  # Assuming PIL for image processing
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        """
        Initializes the custom dataset.

        Args:
            data_dir (str): Path to the directory containing images and labels.
            transform (torchvision.transforms, optional): Transformations to apply to images. Defaults to None.
        """
        self.data_dir = data_dir
        self.image_paths = []
        self.labels = []

        # Process data directory structure (modify based on your data organization)
        for class_name in os.listdir(data_dir):
            class_path = os.path.join(data_dir, class_name)
            if os.path.isdir(class_path):
                for filename in os.listdir(class_path):
                    image_path = os.path.join(class_path, filename)
                    if os.path.isfile(image_path):  # Ensure it's a valid image file
                        self.image_paths.append(image_path)
                        self.labels.append(class_name)  # Assuming class name is the label

        # Check if there are any images loaded
        if len(self.image_paths) == 0:
            raise ValueError("No images found in the data directory.")

        self.transform = transform  # Optional transformation for data augmentation

    def __getitem__(self, index):
        """
        Gets one data pair (image and label) from the dataset.

        Args:
            index (int): Index of the data item to retrieve.

        Returns:
            tuple: A tuple containing the processed image (tensor) and its corresponding label (tensor).
        """
        image_path = self.image_paths[index]
        label = self.labels[index]

        # Load image (modify based on your image format)
        image = Image.open(image_path).convert('RGB')  # Assuming RGB images

        # Apply transformations (if provided)
        if self.transform:
            image = self.transform(image)

        # Convert label to a tensor (modify based on label format)
        label_tensor = torch.tensor(label)  # Assuming class names as labels

        return image, label_tensor

    def __len__(self):
        """
        Returns the total number of data samples in the dataset.
        """
        return len(self.image_paths)

# Create the custom dataset with data directory
custom_dataset = CustomDataset(data_dir="./data")  # Replace with your actual data directory

# Create the data loader
train_loader = DataLoader(custom_dataset, batch_size=32, shuffle=True)

# 6. Pretrained Model

In [27]:
# Download and load the pretrained ResNet-18
resnet = torchvision.models.resnet18(pretrained=True)

c:\Users\immab\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\immab\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [28]:
# if you want to finetune only the top layer of the model, set as below

for param in resnet.parameters():
    param.requires_grad = False

In [29]:
# replace the top layer for finetuning
resnet.fc = nn.Linear(resnet.fc.in_features, 100)

# Forward pass
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print(outputs.shape)

torch.Size([64, 100])


# 7. save and load the model

In [30]:
# save and load the entire model
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

C:\Users\immab\AppData\Local\Temp\ipykernel_496\1556255639.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('model.ckpt')


In [31]:
# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

C:\Users\immab\AppData\Local\Temp\ipykernel_496\250726142.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet.load_state_dict(torch.load('params.ckpt'))


<All keys matched successfully>